In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras

import scipy.io
from sklearn.preprocessing import LabelEncoder

from shutil import copyfile
import zipfile

np.random.seed(42)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
copyfile('/content/drive/MyDrive/Colab Notebooks/training2017.zip', 'training2017.zip')
zipf = zipfile.ZipFile('training2017.zip')
zipf.extractall()
zipf.close()

Mounted at /content/drive


In [3]:
# Storing the output
le = LabelEncoder()
data = pd.read_csv('training2017/REFERENCE.csv', header=None, names=['name', 'label'])
y = le.fit_transform(data.label.values)
y = y.reshape(-1, 1)

In [4]:
# Storing the input
def get_data(file):
    data = scipy.io.loadmat(file)['val'][0]
    return data.astype(float)

data_list = [get_data('training2017/{}.mat'.format(i)) for i in data.name.values]



In [5]:
input_size = 9000
data_list_resized = []
labels_resized = []
for elem, label in zip(data_list, y):
  num_sub_arrays = len(elem) // input_size
  if num_sub_arrays != 0:
    temp = np.split(elem[:num_sub_arrays * input_size], num_sub_arrays)
    for elem2 in temp:
      data_list_resized.append(elem2)
      labels_resized.append(label[0])

X_resized = np.array(data_list_resized)
y_resized = np.array(labels_resized)


In [6]:
y_resized=keras.utils.to_categorical(y_resized)

In [7]:
inputs = keras.layers.Input(shape=(9000, 1))
x = keras.layers.LSTM(10)(inputs)
outputs = keras.layers.Dense(4, activation='softmax')(x)

net = keras.Model(inputs,outputs)

In [8]:
net.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9000, 1)]         0         
                                                                 
 lstm (LSTM)                 (None, 10)                480       
                                                                 
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 524 (2.05 KB)
Trainable params: 524 (2.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
net.compile(loss="categorical_crossentropy",
            optimizer="Adam", run_eagerly=True)


In [11]:
history = net.fit(x= X_resized, y=y_resized, epochs=1, batch_size = 128, shuffle=False)

66/66 [==============================] - 24s 308ms/step - loss: 1.4162
